In [18]:
from distutils import extension
import json
import re
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup

URL_BASE = "https://www.indeed.com"


def main():
    cidade = input("Nome da cidade:")

    keywords = input("Palavras-chaves da vaga:")

    npaginas = input("Selecione o número de páginas para análise:")

    cidade = '+'.join(cidade.split(' '))
    keywords = '+'.join(keywords.split(' '))
    url_scrape = URL_BASE + "/jobs?q=" + keywords + "&l=" + cidade
    npaginas = int(npaginas)
    dados = data(url_scrape, npaginas)
    with open('data.json', 'w') as fp:
        json.dump(dados, fp, sort_keys=True, indent=4, ensure_ascii=False)


def data(url_scrape, npaginas):
    data = []
    for i in range(0, npaginas):
        extension = ""
        if i != 0:
            extension = "&start" + str(i*10)
        url = url_scrape + extension
        req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        web_page = urlopen(req).read()
        soup = BeautifulSoup(web_page, 'html.parser')
        data = get_data_from_webpage(data, soup)
    return data


def extracao(job, div):
    for a in div.findAll('a', attrs={'class': 'jobtitle turnstileLink'}):
        job['Vaga'] = a['Vaga']
    for a1 in div.findAll('a', attrs={'data-tn-element': 'companyName'}):
        job['Empresa'] = a1.text.strip()
    for span in div.findAll('span', attrs={'class': 'ratingsContent'}):
        job['Avaliação'] = span.text.strip()
    for span1 in div.findAll('span', attrs={'class': 'location acessible-contrast-color-location'}):
        job['Localização'] = span1.text.strip()
    for div1 in div.findAll('div', attrs={'class': 'summary'}):
        resumo = div1.text.strip()
        job['Resumo'] = re.sub(' +', ' ', resumo.replace("\n", ""))
    for span2 in div.findAll('span', attrs={'class': 'date'}):
        job['Data'] = span2.text.strip()
    return job


def get_data_from_webpage(data, soup):
    job_posts = []
    for div in soup.findAll('div', attrs={'class': 'jobsearch-SerpJobCard unifiedRow row result'}):
        job = dict()
        job = extracao(job, div)
        job_posts.append(div['data-jk'])
        single_job_post_extension_url = "https://www.indeed.com/viewjob?jk=" + \
            div['data-jk']
        job['url'] = single_job_post_extension_url
        req = Request(single_job_post_extension_url,
                      headers={'User-Agent': 'Mozilla/5.0'})
        web_page = urlopen(req).read()
        job_soup = BeautifulSoup(web_page, 'html.parser')

        for inside_div in job_soup.findAll('div', attrs={'class': 'jobsearch-jobDescriptionText'}):
            details = inside_div.text.strip()[:100] + "..."
            job['Detalhes'] = re.sub(' +', ' ', details.replace("\n", ""))
        data.append(job)

        return data


if __name__ == "__main__":
    main()
    print('Extração completa, verifique arquivo json')


UnicodeEncodeError: 'ascii' codec can't encode character '\xe3' in position 20: ordinal not in range(128)